# CS 5542 — Lab 4 Notebook (Team Project)
## RAG Application Integration, Deployment, and Monitoring (Deadline: Feb. 12, 2026)

**Purpose:** This notebook is a **project-aligned template** for Lab 4. Your team should reuse your Lab-3 multimodal RAG pipeline and integrate it into a **deployable application** with **automatic logging** and **failure analysis**.

### Submission policy
- **Survey:** submitted **individually**
- **Deliverables (GitHub repo / notebook / report / deployment link):** submitted **as a team**

### Team-size requirement
- **1–2 students:** Base requirements + **1 extension**
- **3–4 students:** Base requirements + **2–3 extensions**

---

## What you will build (at minimum)
1. A **Streamlit app** that accepts a question and returns:
   - an **answer**
   - **retrieved evidence** with citations
   - **metrics panel** (latency, P@5, R@10 if applicable)
2. An **automatic logger** that appends to: `logs/query_metrics.csv`
3. A **mini gold set** of **5 project queries** (Q1–Q5) for evaluation
4. **Two failure cases** with root cause + proposed fix

> **Important:** Lab 4 focuses on **application integration and deployment**, not on redesigning retrieval. Prefer reusing your Lab-3 modules.

---

## Recommended repository structure (for your team repo)
```
/app/              # Streamlit UI (required)
/rag/              # Retrieval + indexing modules (reuse from Lab 3)
/logs/             # query_metrics.csv (auto-created)
/data/             # your project-aligned PDFs/images (do NOT commit large/private data)
/api/              # optional FastAPI backend (extension)
/notebooks/        # this notebook
requirements.txt
README.md
```

---

## Contents of this notebook
1. Setup & environment checks  
2. Project dataset wiring (connect your Lab-3 ingestion)  
3. Mini gold set (Q1–Q5)  
4. Retrieval + answer function (reuse your Lab-3 pipeline)  
5. Evaluation + logging (required)  
6. Streamlit app skeleton (required)  
7. Optional extension: FastAPI backend  
8. Deployment checklist + failure analysis template


In [ ]:
# Task: Load provided demo files (docs + images)
# Place Your_zip_files.zip next to this notebook or upload it to the runtime.
import os, zipfile
from pathlib import Path

ZIP_NAME = 'Your_zip_files.zip'
zip_candidates = [
    ZIP_NAME,
    f'/mnt/data/{ZIP_NAME}',
    f'./{ZIP_NAME}',
]

zip_path = None
for p in zip_candidates:
    if os.path.exists(p):
        zip_path = p
        break

if zip_path is None:
    print('⚠️ Demo ZIP not found. If running in Colab/Kaggle, upload:', ZIP_NAME)
else:
    with zipfile.ZipFile(zip_path, 'r') as z:
        z.extractall('.')
    print('✅ Extracted demo files from:', zip_path)

# Expected folder structure after extraction:
# ./data/docs/*.txt
# ./data/images/*.(png|jpg)
print('Docs folder exists:', os.path.isdir('./data/docs'))
print('Images folder exists:', os.path.isdir('./data/images'))

if os.path.isdir('./data/docs'):
    print('Sample docs:', sorted(os.listdir('./data/docs'))[:6])
if os.path.isdir('./data/images'):
    print('Sample images:', sorted(os.listdir('./data/images'))[:6])


In [ ]:
# Ensure a numeric/table-like demo file exists for Q4
import os
numeric_path = './data/docs/07_numeric_table.txt'
if not os.path.exists(numeric_path):
    with open(numeric_path, 'w', encoding='utf-8') as f:
        f.write(
            'Fusion Hyperparameters (Table 1)\n'
            'alpha = 0.50\n'
            'top_k = 5\n'
            'missing_evidence_score_threshold = 0.05\n'
            'latency_alert_ms = 2000\n'
        )
    print('✅ Created:', numeric_path)
else:
    print('✅ Numeric demo file already present:', numeric_path)


In [ ]:
# Sanity checks: ensure demo docs are loaded
import os, glob
doc_files = glob.glob('./data/docs/*.txt')
print('Found .txt docs:', len(doc_files))
assert len(doc_files) > 0, 'No docs found. Ensure the demo ZIP was extracted and ./data/docs exists.'

# Preview one document
with open(doc_files[0], 'r', encoding='utf-8') as f:
    preview = f.read()[:600]
print('Preview:', os.path.basename(doc_files[0]))
print(preview)


In [ ]:
# Load demo documents into a single list used throughout the notebook
import glob, os

DOC_DIR = './data/docs'
doc_files = sorted(glob.glob(os.path.join(DOC_DIR, '*.txt')))
if len(doc_files) == 0:
    raise RuntimeError('No .txt documents found in ./data/docs. Ensure the demo ZIP was extracted.')

documents = []
for p in doc_files:
    with open(p, 'r', encoding='utf-8') as f:
        txt = f.read().strip()
    if not txt:
        continue
    doc_id = os.path.basename(p)
    documents.append({'doc_id': doc_id, 'source': p, 'text': txt})

print('✅ Loaded documents:', len(documents))
print('Example doc_id:', documents[0]['doc_id'])
print(documents[0]['text'][:300])


In [ ]:
# Load demo images and create lightweight text surrogates (captions) for multimodal retrieval
import glob, os

IMG_DIR = './data/images'
img_files = sorted(glob.glob(os.path.join(IMG_DIR, '*.*')))
img_files = [p for p in img_files if p.lower().endswith(('.png','.jpg','.jpeg','.webp'))]

# Minimal captions so images participate in retrieval without requiring a vision encoder
IMAGE_CAPTIONS = {
    'rag_pipeline.png': 'RAG pipeline diagram: ingest, chunk, index, retrieve top-k evidence, build context, generate grounded answer, log metrics for monitoring.',
    'retrieval_modes.png': 'Retrieval modes diagram: BM25 keyword, vector semantic, hybrid fusion, multi-hop hop-1 to hop-2 refinement.',
}

images = []
for p in img_files:
    fid = os.path.basename(p)
    cap = IMAGE_CAPTIONS.get(fid, fid.replace('_',' ').replace('.png','').replace('.jpg',''))
    images.append({'img_id': fid, 'source': p, 'text': cap})

print('✅ Loaded images:', len(images))
if images:
    print('Example image:', images[0]['img_id'])
    print('Caption:', images[0]['text'])

# Unified evidence store used by retrieval (text + images)
items = []
for d in documents:
    items.append({
        'evidence_id': d.get('doc_id') or os.path.basename(d.get('source','')),
        'modality': 'text',
        'source': d.get('source'),
        'text': d.get('text','')
    })
for im in images:
    items.append({
        'evidence_id': f"img::{im['img_id']}",
        'modality': 'image',
        'source': im.get('source'),
        'text': im.get('text','')
    })

assert len(items) > 0, 'Evidence store is empty.'
print('✅ Unified evidence items:', len(items), '(text:', len(documents), ', images:', len(images), ')')


# 1) Setup & environment checks

This notebook includes **safe defaults** and **lightweight code examples**.  
Replace the placeholder pieces with your Lab-3 implementation (PDF parsing, OCR, multimodal evidence, hybrid retrieval, reranking).

### Install dependencies (edit as needed)
- Core: `streamlit`, `pandas`, `numpy`, `requests`
- Optional: `fastapi`, `uvicorn` (if you do the FastAPI extension)
- Retrieval examples: `scikit-learn` (TF-IDF baseline), optionally `sentence-transformers` (dense embeddings)

> In your team repo, always keep a clean `requirements.txt` for reproducibility.


In [ ]:
# If running in Colab or fresh environment, uncomment installs:
# !pip -q install streamlit pandas numpy requests scikit-learn
# # Optional (FastAPI extension):
# !pip -q install fastapi uvicorn pydantic
# # Optional (dense retrieval):
# !pip -q install sentence-transformers

import os, json, time
from pathlib import Path
import pandas as pd
import numpy as np

print("Python OK. Working directory:", os.getcwd())


# 2) Project paths + configuration

Set your project data paths and key parameters here.

- Do **not** hardcode secrets (API keys) in notebooks or repos.
- If you use a hosted LLM, read from environment variables locally.

**Tip:** Keep these settings mirrored in `rag/config.py` so your Streamlit app uses the same config.


In [ ]:
from dataclasses import dataclass

@dataclass
class Lab4Config:
    project_name: str = "YOUR_PROJECT_NAME"
    data_dir: str = "./data"        # where your PDFs/images live locally
    logs_dir: str = "./logs"
    log_file: str = "./logs/query_metrics.csv"
    top_k_default: int = 10
    eval_p_at: int = 5
    eval_r_at: int = 10

cfg = Lab4Config()
Path(cfg.logs_dir).mkdir(parents=True, exist_ok=True)
print(cfg)


# 3) Dataset wiring (project-aligned)

For Lab 4, your **data, application UI, and models** must be aligned to your team project.

## Required (project-aligned)
- 2–6 PDFs
- 5–15 images/figures/tables (if your project is multimodal)

## In Lab 3 you likely had:
- PDF text extraction (PyMuPDF)
- OCR / captions for figures or scanned pages
- Chunking + indexing (dense/sparse/hybrid)
- Reranking (optional)
- Grounded answer generation with citations

### What to do here
1. Point this notebook to your dataset folder.
2. Load *already-prepared* chunks/evidence from Lab 3 (recommended), OR
3. Call your Lab-3 ingestion function to rebuild the index.

Below is a **minimal example** that loads plain text files as “documents” so the notebook is runnable even without PDFs.
Replace it with your Lab-3 ingestion code.


In [ ]:
# Minimal runnable loader (replace with your Lab-3 ingestion + chunking)
# Expected structure (example):
# ./data/
#   docs/
#     doc1.txt
#     doc2.txt
#
# For PDFs/images, reuse your Lab-3 ingestion + chunking and store chunks as JSONL/CSV.

from pathlib import Path

docs_dir = Path(cfg.data_dir) / "docs"
docs_dir.mkdir(parents=True, exist_ok=True)

# Create a tiny demo corpus if empty (so the notebook runs)
demo_file = docs_dir / "demo_doc.txt"
if not any(docs_dir.glob("*.txt")):
    demo_file.write_text(
        "This is a demo document for Lab 4. Replace this with your project PDFs and evidence chunks.\n"
        "Key idea: retrieval quality drives grounded answers. Provide citations for all claims.\n"
        "If missing evidence, return: Not enough evidence in the retrieved context.\n",
        encoding="utf-8"
    )

def load_text_docs(docs_path: Path):
    items = []
    for p in sorted(docs_path.glob("*.txt")):
        items.append({"doc_id": p.stem, "source": str(p), "text": p.read_text(errors='ignore')})
    return items

documents = load_text_docs(docs_dir)
print("Loaded docs:", len(documents))
documents[0].keys(), documents[0]["doc_id"]


# 4) Mini Gold Set (Q1–Q5) — Required

Create **5 project-relevant queries** and define a simple evidence rubric.

- **Q1–Q3:** typical project queries (answerable using evidence)
- **Q4:** multimodal evidence query (table/figure heavy, OCR/captions should help)
- **Q5:** missing-evidence or ambiguous query (must trigger safe behavior)

For each query, define:
- `gold_evidence_ids`: list of evidence identifiers that are relevant (doc_id/page/fig id)
- `answer_criteria`: 1–2 bullets
- `citation_format`: how you will cite (e.g., `[Doc1 p3]`, `[fig2]`)

This enables **consistent evaluation** and makes logging meaningful.


In [ ]:
# Task: Populate a mini gold set for monitoring and ablation
# Evidence identifiers use doc_ids from the demo corpus (file basenames under ./data/docs).

mini_gold = [
    {
        "query_id": "Q1",
        "question": "What is Retrieval-Augmented Generation (RAG) and what does grounding mean?",
        "gold_evidence_ids": ["01_rag_overview.txt"],
        "answer_criteria": ["Defines RAG", "Explains grounding", "Includes a citation"],
        "citation_format": "[doc_id]"
    },
    {
        "query_id": "Q2",
        "question": "If the evidence is insufficient, what should the system say?",
        "gold_evidence_ids": ["05_missing_evidence_policy.txt"],
        "answer_criteria": ["Returns the missing-evidence phrase", "Includes a citation"],
        "citation_format": "[doc_id]"
    },
    {
        "query_id": "Q3",
        "question": "Why would you use hybrid retrieval instead of only BM25 or only vectors?",
        "gold_evidence_ids": ["02_hybrid_retrieval.txt"],
        "answer_criteria": ["Mentions BM25 strengths", "Mentions vector strengths", "Explains fusion", "Includes a citation"],
        "citation_format": "[doc_id]"
    },
    {
        "query_id": "Q4",
        "question": "From Table 1, what is the value of alpha used for fusion?",
        "gold_evidence_ids": ["07_numeric_table.txt"],
        "answer_criteria": ["Extracts the numeric value 0.50", "Includes a citation"],
        "citation_format": "[doc_id]"
    },
    {
        "query_id": "Q5",
        "question": "Who won the FIFA World Cup in 2050?",
        "gold_evidence_ids": ["N/A"],
        "answer_criteria": ["Returns the missing-evidence phrase", "No hallucination"],
        "citation_format": ""
    },
]

pd.DataFrame(mini_gold)[["query_id","question","gold_evidence_ids"]]



# Optional multimodal query (image evidence via caption surrogate)
mini_gold.append({
    "query_id": "Q6",
    "question": "Which retrieval modes are shown in the retrieval modes diagram?",
    "gold_evidence_ids": ["img::retrieval_modes.png"],
    "answer_criteria": ["Mentions BM25, vector, hybrid, multi-hop"],
    "citation_format": "[evidence_id]"
})



In [ ]:
# Task: Mini gold set (evidence IDs) for evaluation
# Evidence IDs refer to demo files under ./data/docs (use basenames). Image evidence uses the prefix img::
import pandas as pd

mini_gold = [
    {
        'query_id': 'Q1',
        'question': 'What is Retrieval-Augmented Generation (RAG) and what does grounding mean?',
        'gold_evidence_ids': ['01_rag_overview.txt']
    },
    {
        'query_id': 'Q2',
        'question': 'If the evidence is insufficient, what should the system say?',
        'gold_evidence_ids': ['05_missing_evidence_policy.txt']
    },
    {
        'query_id': 'Q3',
        'question': 'Why would you use hybrid retrieval instead of only BM25 or only vectors?',
        'gold_evidence_ids': ['02_hybrid_retrieval.txt']
    },
    {
        'query_id': 'Q4',
        'question': 'From Table 1, what is the value of alpha used for fusion?',
        'gold_evidence_ids': ['07_numeric_table.txt']
    },
    {
        'query_id': 'Q5',
        'question': 'Who won the FIFA World Cup in 2050?',
        'gold_evidence_ids': ['N/A']
    },
    {
        'query_id': 'Q6',
        'question': 'Which retrieval modes are shown in the retrieval modes diagram?',
        'gold_evidence_ids': ['img::retrieval_modes.png']
    },
]

pd.DataFrame(mini_gold)[['query_id','question','gold_evidence_ids']]


# 5) Retrieval + Answer Function (Reuse Lab 3)

Below is a **baseline TF‑IDF retriever** so this notebook is runnable.
Replace with your Lab-3 retrieval stack:
- dense (SentenceTransformers + FAISS/Chroma)
- sparse (BM25)
- hybrid fusion
- optional reranking

### Required output contract (recommended)
Your retrieval function should return a list of evidence items:
- `chunk_id` or `doc_id`
- `source`
- `score`
- `citation_tag` (e.g., `[Doc1 p3]`, `[fig2]`)
- `text` (the evidence text shown to users)

Your answer function must enforce:
- **Citations for claims**
- If missing evidence: **return exactly**  
  `Not enough evidence in the retrieved context.`


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Build a simple TF-IDF index over documents (demo baseline)
corpus = [d["text"] for d in documents]
doc_ids = [d["doc_id"] for d in documents]
sources = [d["source"] for d in documents]

vectorizer = TfidfVectorizer(stop_words="english")
X = vectorizer.fit_transform(corpus)

def retrieve_tfidf(question: str, top_k: int = 5):
    q = vectorizer.transform([question])
    sims = cosine_similarity(q, X).ravel()
    idxs = np.argsort(-sims)[:top_k]
    evidence = []
    for rank, i in enumerate(idxs):
        evidence.append({
            "chunk_id": doc_ids[i],
            "source": sources[i],
            "score": float(sims[i]),
            "citation_tag": f"[{doc_ids[i]}]",
            "text": corpus[i][:800]  # truncate for UI
        })
    return evidence

MISSING_EVIDENCE_MSG = "Not enough evidence in the retrieved context."

def generate_answer_stub(question: str, evidence: list):
    """Replace with your LLM/VLM generation.
    For this template we produce a simple grounded response.
    """
    if not evidence or max(e.get("score", 0.0) for e in evidence) < 0.05:
        return MISSING_EVIDENCE_MSG

    # Minimal grounded "answer" example: summarize top evidence
    top = evidence[0]
    answer = (
        f"Based on the retrieved evidence {top['citation_tag']}, "
        f"the system should ground its response in retrieved context and cite sources. "
        f"If evidence is missing, it must respond with: '{MISSING_EVIDENCE_MSG}'. "
        f"{top['citation_tag']}"
    )
    return answer

# Quick test
test_q = mini_gold[0]["question"]
ev = retrieve_tfidf(test_q, top_k=3)
print("Top evidence:", ev[0]["chunk_id"], ev[0]["score"])
print("Answer:", generate_answer_stub(test_q, ev))


# 6) Evaluation + Logging (Required)

Every query must append to: `logs/query_metrics.csv`

Required columns (minimum):
- timestamp
- query_id
- retrieval_mode
- top_k
- latency_ms
- Precision@5
- Recall@10
- evidence_ids_returned
- faithfulness_pass
- missing_evidence_behavior

> If your gold set is incomplete (common for Q4/Q5), compute P/R only for labeled queries and still log latency/evidence IDs.

## How we define metrics (simple)
- `Precision@K`: (# retrieved evidence IDs in gold) / K
- `Recall@K`: (# retrieved evidence IDs in gold) / (size of gold set)

**Faithfulness (Yes/No):**
- Yes if the answer **only** uses retrieved evidence and includes citations.
- For this template, we implement a simple heuristic. Replace with your rubric/judge if desired.


In [ ]:
import os
import csv
from datetime import datetime, timezone



def _canon_evidence_id(x: str) -> str:
    x = str(x).strip()
    # keep img:: prefix intact
    if x.startswith('img::'):
        return x
    # normalize file ids: allow with/without extension
    if x.endswith('.txt'):
        return x[:-4]
    return x

def _normalize_retrieved_ids(retrieved):
    """Normalize retrieved outputs into a list of evidence IDs.
    Returns canonical IDs (doc_id without .txt, or img::filename).

    Supports: list[dict], list[(idx,score)], list[str].
    """
    if retrieved is None:
        return []
    if len(retrieved) == 0:
        return []
    # list[str]
    if isinstance(retrieved[0], str):
        return [_canon_evidence_id(r) for r in retrieved]
    # list[dict]
    if isinstance(retrieved[0], dict):
        out=[]
        for r in retrieved:
            if 'evidence_id' in r and r['evidence_id']:
                out.append(_canon_evidence_id(r['evidence_id']))
            elif 'doc_id' in r and r['doc_id']:
                out.append(_canon_evidence_id(r['doc_id']))
            elif 'source' in r and r['source']:
                out.append(_canon_evidence_id(os.path.basename(str(r['source']))))
        return out
    # list[(idx, score)]
    if isinstance(retrieved[0], (tuple, list)) and len(retrieved[0]) >= 1:
        out=[]
        for item in retrieved:
            idx = int(item[0])
            if 'items' in globals() and 0 <= idx < len(items):
                out.append(_canon_evidence_id(items[idx].get('evidence_id')))
            elif 'documents' in globals() and 0 <= idx < len(documents):
                out.append(_canon_evidence_id(documents[idx].get('doc_id') or os.path.basename(documents[idx].get('source',''))))
        return out
    return []

def _normalize_gold_ids(gold_ids):
    if not gold_ids or gold_ids == ['N/A']:
        return None
    return [_canon_evidence_id(g) for g in gold_ids]

def precision_at_k(retrieved, gold_ids, k):
    gold = _normalize_gold_ids(gold_ids)
    if gold is None:
        return None
    retrieved_ids = _normalize_retrieved_ids(retrieved)[:k]
    if k == 0:
        return None
    return len(set(retrieved_ids) & set(gold)) / float(k)

def recall_at_k(retrieved, gold_ids, k):
    gold = _normalize_gold_ids(gold_ids)
    if gold is None:
        return None
    retrieved_ids = _normalize_retrieved_ids(retrieved)[:k]
    denom = float(len(set(gold)))
    return (len(set(retrieved_ids) & set(gold)) / denom) if denom > 0 else None



def faithfulness_heuristic(answer: str, evidence: list):
    # Simple heuristic: answer includes at least one citation tag from evidence OR is missing-evidence msg
    if answer.strip() == MISSING_EVIDENCE_MSG:
        return True
    tags = [e["citation_tag"] for e in evidence[:5]]
    return any(tag in answer for tag in tags)

def missing_evidence_behavior(answer: str, evidence: list):
    # Pass if either: evidence present and answer not missing-evidence; or evidence absent and answer is missing-evidence msg
    has_ev = bool(evidence) and max(e.get("score", 0.0) for e in evidence) >= 0.05
    if not has_ev:
        return "Pass" if answer.strip() == MISSING_EVIDENCE_MSG else "Fail"
    else:
        return "Pass" if answer.strip() != MISSING_EVIDENCE_MSG else "Fail"

def ensure_logfile(path: str, header: list):
    p = Path(path)
    p.parent.mkdir(parents=True, exist_ok=True)
    if not p.exists():
        with open(p, "w", newline="", encoding="utf-8") as f:
            writer = csv.writer(f)
            writer.writerow(header)

LOG_HEADER = [
    "timestamp", "query_id", "retrieval_mode", "top_k", "latency_ms",
    "Precision@5", "Recall@10",
    "evidence_ids_returned", "gold_evidence_ids",
    "faithfulness_pass", "missing_evidence_behavior"
]
ensure_logfile(cfg.log_file, LOG_HEADER)

def run_query_and_log(query_item, retrieval_mode = 'hybrid', top_k=10):
    question = query_item["question"]
    gold_ids = query_item.get("gold_evidence_ids", [])

    t0 = time.time()
    evidence = retrieve_tfidf(question, top_k=top_k)  # replace with your pipeline + modes
    answer = generate_answer_stub(question, evidence) # replace with LLM/VLM
    latency_ms = (time.time() - t0) * 1000.0

    retrieved_ids = [e["chunk_id"] for e in evidence]
    p5 = precision_at_k(retrieved_ids, gold_ids, cfg.eval_p_at) if gold_ids else np.nan
    r10 = recall_at_k(retrieved_ids, gold_ids, cfg.eval_r_at) if gold_ids else np.nan

    faithful = faithfulness_heuristic(answer, evidence)
    meb = missing_evidence_behavior(answer, evidence)

    row = [
        datetime.now(timezone.utc).isoformat(),
        query_item["query_id"],
        retrieval_mode,
        top_k,
        round(latency_ms, 2),
        p5,
        r10,
        json.dumps(retrieved_ids),
        json.dumps(gold_ids),
        "Yes" if faithful else "No",
        meb
    ]
    with open(cfg.log_file, "a", newline="", encoding="utf-8") as f:
        writer = csv.writer(f)
        writer.writerow(row)

    return {"answer": answer, "evidence": evidence, "p5": p5, "r10": r10, "latency_ms": latency_ms, "faithful": faithful, "meb": meb}

# Run all five queries once (demo)
results = []
for qi in mini_gold:
    results.append(run_query_and_log(qi, retrieval_mode = 'hybrid', top_k=cfg.top_k_default))

pd.read_csv(cfg.log_file).tail(8)


In [ ]:
# Task: Run retrieval + answer generation for all mini-gold queries
# This cell is self-contained: if retrieval/indexing cells were skipped, it will bootstrap a TF-IDF retriever.
import os
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Build a local evidence list if not already present
if 'items' in globals():
    _evidence = items
elif 'documents' in globals():
    _evidence = []
    for d in documents:
        _evidence.append({
            'evidence_id': d.get('doc_id') or os.path.basename(d.get('source','')),
            'modality': 'text',
            'source': d.get('source'),
            'text': d.get('text','')
        })
else:
    raise NameError('Neither items nor documents are defined. Run the ZIP extraction + document loading cells first.')

assert len(_evidence) > 0, 'Evidence store is empty.'

# Canonicalize evidence ids for consistent evaluation
def _canon_evidence_id(x: str) -> str:
    x = str(x).strip()
    if x.startswith('img::'):
        return x
    return x[:-4] if x.endswith('.txt') else x

# Bootstrap TF-IDF retriever if no retriever exists
if 'retrieve_hybrid' not in globals() and 'retrieve_tfidf' not in globals() and 'retrieve' not in globals():
    _texts = [it.get('text','') for it in _evidence]
    _tfidf = TfidfVectorizer(stop_words=None, token_pattern=r'(?u)\b\w+\b')
    _tfidf_mat = _tfidf.fit_transform(_texts)

    def retrieve_tfidf(query, top_k=10):
        qv = _tfidf.transform([query])
        sims = cosine_similarity(qv, _tfidf_mat).ravel()
        idx = np.argsort(sims)[::-1][:top_k]
        return [(int(i), float(sims[i])) for i in idx]

# Define retrieve() wrapper if missing
if 'retrieve' not in globals():
    def retrieve(question, retrieval_mode='hybrid', top_k=10, alpha=0.6):
        # Prefer hybrid if available; otherwise TF-IDF
        if retrieval_mode == 'hybrid' and 'retrieve_hybrid' in globals():
            hits = retrieve_hybrid(question, top_k=top_k, alpha=alpha)
            return hits, {'mode':'hybrid'}
        if 'retrieve_tfidf' in globals():
            hits = retrieve_tfidf(question, top_k=top_k)
            return hits, {'mode':'tfidf'}
        raise NameError('No retriever available. Execute the retrieval/indexing section.')

# Ensure build_context exists
if 'build_context' not in globals():
    def build_context(hit_ids, max_chars=1400):
        parts=[]
        for i in hit_ids:
            parts.append(f"[{_evidence[i].get('evidence_id')}] {_evidence[i].get('text','')}")
        ctx='\n'.join(parts)
        return ctx[:max_chars]

# Ensure extractive_answer exists
if 'extractive_answer' not in globals():
    import re
    def extractive_answer(query, context):
        q=set(re.findall(r'[A-Za-z]+', query.lower()))
        sents=re.split(r'(?<=[.!?])\s+', (context or '').strip())
        scored=[]
        for s in sents:
            w=set(re.findall(r'[A-Za-z]+', s.lower()))
            scored.append((len(q & w), s.strip()))
        scored.sort(key=lambda x:x[0], reverse=True)
        best=[s for sc,s in scored[:3] if sc>0]
        return ' '.join(best) if best else 'Not enough information in the context.'

rows=[]
for ex in mini_gold:
    qid = ex.get('query_id')
    question = ex.get('question')
    gold = ex.get('gold_evidence_ids')

    if 'run_query_and_log' in globals():
        # Call run_query_and_log with the full query item dictionary 'ex'
        out = run_query_and_log(ex, retrieval_mode='hybrid', top_k=10)
        answer = out.get('answer')
        # The 'evidence' key from run_query_and_log output contains a list of dicts with 'chunk_id'
        evidence = [e['chunk_id'] for e in out.get('evidence', [])]
    else:
        hits, debug = retrieve(question, retrieval_mode='hybrid', top_k=10)
        hit_ids = [int(i) for i,_ in hits]
        context = build_context(hit_ids[:10])
        answer = extractive_answer(question, context)
        evidence = [_canon_evidence_id(_evidence[i].get('evidence_id')) for i in hit_ids[:10]]

    rows.append({
        'query_id': qid,
        'question': question,
        'answer': answer,
        'evidence_ids_returned(top10)': evidence,
        'gold_evidence_ids': gold,
    })

df_answers = pd.DataFrame(rows)
df_answers


# 7) Streamlit App Skeleton (Required)

You will create a Streamlit app file in your repo, e.g.:

- `app/main.py`

This notebook can generate a starter `app/main.py` for your team.

### Required UI components
- Query input box
- Retrieval controls (mode, top_k, multimodal toggle if applicable)
- Answer panel
- Evidence panel (with citations)
- Metrics panel (latency, P@5, R@10 if available)
- Logging happens automatically on each query

> This skeleton calls functions in your Python modules. Prefer moving retrieval logic into `/rag/` and importing it.


In [ ]:
# Generate a starter Streamlit app file (edit paths as needed).
# In your repo: create /app/main.py and move shared logic into /rag/

streamlit_code = r'''
import json, time
from pathlib import Path
import streamlit as st
import pandas as pd

# --- Import your team pipeline here ---
# from rag.pipeline import retrieve, generate_answer, run_query_and_log

MISSING_EVIDENCE_MSG = "Not enough evidence in the retrieved context."

st.set_page_config(page_title="CS5542 Lab 4 — Project RAG App", layout="wide")
st.title("CS 5542 Lab 4 — Project RAG Application")
st.caption("Project-aligned Streamlit UI + automatic logging + failure monitoring")

# Sidebar controls
st.sidebar.header("Retrieval Settings")
retrieval_mode = st.sidebar.selectbox("retrieval_mode", ["tfidf", "dense", "sparse", "hybrid", "hybrid_rerank"])
top_k = st.sidebar.slider("top_k", min_value=1, max_value=30, value=10, step=1)
use_multimodal = st.sidebar.checkbox("use_multimodal", value=True)

st.sidebar.header("Logging")
log_path = st.sidebar.text_input("log file", value="logs/query_metrics.csv")

# --- Mini gold set (replace with your team's Q1–Q5) ---
# Tip: keep the same structure as in your Lab 4 notebook so IDs match logs.
MINI_GOLD = {
    "Q1": {"question": "Replace with your project Q1", "gold_evidence_ids": []},
    "Q2": {"question": "Replace with your project Q2", "gold_evidence_ids": []},
    "Q3": {"question": "Replace with your project Q3", "gold_evidence_ids": []},
    "Q4": {"question": "Replace with your project Q4 (multimodal/table/figure)", "gold_evidence_ids": []},
    "Q5": {"question": "Replace with your project Q5 (missing-evidence case)", "gold_evidence_ids": []},
}

st.sidebar.header("Evaluation")
query_id = st.sidebar.selectbox("query_id (for logging)", list(MINI_GOLD.keys()))
use_gold_question = st.sidebar.checkbox("Use the gold-set question text", value=True)

# Main query
default_q = MINI_GOLD[query_id]["question"] if use_gold_question else ""
question = st.text_area("Enter your question", value=default_q, height=120)
run_btn = st.button("Run Query")

colA, colB = st.columns([2, 1])

def ensure_logfile(path: str):
    p = Path(path)
    p.parent.mkdir(parents=True, exist_ok=True)
    if not p.exists():
        df = pd.DataFrame(columns=[
            "timestamp","query_id","retrieval_mode","top_k","latency_ms",
            "Precision@5","Recall@10","evidence_ids_returned","gold_evidence_ids",
            "faithfulness_pass","missing_evidence_behavior"
        ])
        df.to_csv(p, index=False)

def precision_at_k(retrieved_ids, gold_ids, k=5):
    if not gold_ids:
        return None
    topk = retrieved_ids[:k]
    hits = sum(1 for x in topk if x in set(gold_ids))
    return hits / k

def recall_at_k(retrieved_ids, gold_ids, k=10):
    if not gold_ids:
        return None
    topk = retrieved_ids[:k]
    hits = sum(1 for x in topk if x in set(gold_ids))
    return hits / max(1, len(gold_ids))

# ---- Placeholder demo logic (replace with imports from your /rag module) ----
def retrieve_demo(q: str, top_k: int):
    return [{"chunk_id":"demo_doc","citation_tag":"[demo_doc]","score":0.9,"source":"data/docs/demo_doc.txt","text":"demo evidence..."}]

def answer_demo(q: str, evidence: list):
    if not evidence:
        return MISSING_EVIDENCE_MSG
    return f"Grounded answer using {evidence[0]['citation_tag']} {evidence[0]['citation_tag']}"

def log_row(path: str, row: dict):
    ensure_logfile(path)
    df = pd.read_csv(path)
    df = pd.concat([df, pd.DataFrame([row])], ignore_index=True)
    df.to_csv(path, index=False)
# --------------------------------------------------------------------------

if run_btn and question.strip():
    t0 = time.time()
    evidence = retrieve_demo(question, top_k=top_k)
    answer = answer_demo(question, evidence)
    latency_ms = round((time.time() - t0)*1000, 2)

    retrieved_ids = [e["chunk_id"] for e in evidence]
    gold_ids = MINI_GOLD[query_id].get("gold_evidence_ids", [])

    p5 = precision_at_k(retrieved_ids, gold_ids, k=5)
    r10 = recall_at_k(retrieved_ids, gold_ids, k=10)

    with colA:
        st.subheader("Answer")
        st.write(answer)

        st.subheader("Evidence (Top-K)")
        st.json(evidence)

    with colB:
        st.subheader("Metrics")
        st.write({"latency_ms": latency_ms, "Precision@5": p5, "Recall@10": r10})

    # Log the query using the selected Q1–Q5 ID (not ad-hoc)
    row = {
        "timestamp": pd.Timestamp.utcnow().isoformat(),
        "query_id": query_id,
        "retrieval_mode": retrieval_mode,
        "top_k": top_k,
        "latency_ms": latency_ms,
        "Precision@5": p5,
        "Recall@10": r10,
        "evidence_ids_returned": json.dumps(retrieved_ids),
        "gold_evidence_ids": json.dumps(gold_ids),
        "faithfulness_pass": "Yes" if answer != MISSING_EVIDENCE_MSG else "Yes",
        "missing_evidence_behavior": "Pass"  # update with your rule if needed
    }
    log_row(log_path, row)
    st.success(f"Logged {query_id} to CSV.")
'''
app_dir = Path("app")
app_dir.mkdir(parents=True, exist_ok=True)
(app_dir / "main.py").write_text(streamlit_code, encoding="utf-8")
print("Wrote starter Streamlit app to:", app_dir / "main.py")


# 8) Optional Extension — FastAPI Backend (Recommended for larger teams)

If your team selects the **FastAPI extension**, create:
- `api/server.py` with `POST /query`
- Streamlit UI calls the API using `requests.post(...)`

This separation mirrors real production systems:
UI (Streamlit) → API (FastAPI) → Retrieval + LLM services

Below is a minimal FastAPI starter you can generate.


In [ ]:
fastapi_code = r'''
from fastapi import FastAPI
from pydantic import BaseModel
from typing import List, Dict, Any

app = FastAPI(title="CS5542 Lab 4 RAG Backend")

MISSING_EVIDENCE_MSG = "Not enough evidence in the retrieved context."

class QueryIn(BaseModel):
    question: str
    top_k: int = 10
    retrieval_mode: str = "hybrid"
    use_multimodal: bool = True

@app.post("/query")
def query(q: QueryIn) -> Dict[str, Any]:
    # TODO: import your real pipeline:
    # evidence = retrieve(q.question, top_k=q.top_k, mode=q.retrieval_mode, use_multimodal=q.use_multimodal)
    # answer = generate_answer(q.question, evidence)
    evidence = [{"chunk_id":"demo_doc","citation_tag":"[demo_doc]","score":0.9,"source":"data/docs/demo_doc.txt","text":"demo evidence..."}]
    answer = f"Grounded answer using {evidence[0]['citation_tag']} {evidence[0]['citation_tag']}"
    return {
        "answer": answer,
        "evidence": evidence,
        "metrics": {"top_k": q.top_k, "retrieval_mode": q.retrieval_mode},
        "failure_flag": False
    }
'''
api_dir = Path("api")
api_dir.mkdir(parents=True, exist_ok=True)
(api_dir / "server.py").write_text(fastapi_code, encoding="utf-8")
print("Wrote starter FastAPI server to:", api_dir / "server.py")

print("\nRun locally (terminal):")
print("  uvicorn api.server:app --reload --port 8000")


# 9) Deployment checklist (Required)

Choose **one** deployment route and publish the public link in your README:

- HuggingFace Spaces (Streamlit)
- Streamlit Cloud (GitHub-connected)
- Render / Railway (GitHub-connected)

## README must include
1. Public deployment link  
2. How to run locally:
   - `pip install -r requirements.txt`
   - `streamlit run app/main.py`
3. A screenshot of:
   - the UI
   - evidence panel
   - metrics panel
4. Results snapshot:
   - **5 queries × 2 retrieval modes**
5. Failure analysis:
   - 2 failure cases, root cause, proposed fix

---

# 10) Failure analysis template (Required)

Document:
1. **Retrieval failure** (wrong evidence or missed gold evidence)  
2. **Grounding / missing-evidence failure** (safe behavior or citation enforcement)

For each:
- What happened?
- Why did it happen (root cause)?
- What change will you implement next?

You can paste your analysis into your README under **Lab 4 Results**.


# 11) Team checklist (quick)

Before submission, verify:

- [ ] Dataset, UI, and models are **project-aligned**
- [ ] Streamlit app runs locally and shows: answer + evidence + metrics
- [ ] `logs/query_metrics.csv` is auto-created and appended per query
- [ ] Mini gold set Q1–Q5 exists and P@5/R@10 computed when possible
- [ ] Deployed link is public and listed in README
- [ ] Two failure cases documented with fixes
- [ ] `requirements.txt` and run instructions are correct
- [ ] Individual survey submitted by each teammate

---

If you want to go beyond: add an evaluation dashboard, reranking integration, or FastAPI separation (extensions).


In [ ]:
# Verification: retrieval should return non-empty results for the demo queries
test_q = "What is Retrieval-Augmented Generation (RAG) and what does grounding mean?"

try:
    # Try common retrieval function names in the template
    if 'retrieve_tfidf' in globals():
        hits = retrieve_tfidf(test_q, top_k=5)
    elif 'retrieve' in globals():
        hits = retrieve(test_q, top_k=5)
    else:
        hits = []
    # Normalize hits to a list
    if hits is None:
        hits = []
    # Some functions return list of dicts; some return (idx,score)
    n = len(hits) if hasattr(hits, '__len__') else 0
    print('Demo retrieval hits:', n)
    assert n > 0, 'Retrieval returned empty results. Check indexing cell and corpus construction.'
except Exception as e:
    print('⚠️ Retrieval verification could not run (template function names differ).')
    print('Reason:', type(e).__name__, str(e)[:180])



## GitHub Deployment Example

### Step 1 — Push to GitHub
```bash
git init
git add .
git commit -m "Lab4 deployment"
git branch -M main
git remote add origin https://github.com/<username>/<repo>.git
git push -u origin main
```

### Step 2 — Deploy using Streamlit Cloud
1. Visit https://share.streamlit.io
2. Click **New App**
3. Select your GitHub repository
4. Branch: `main`
5. App path: `app/main.py`
6. Click **Deploy**

### Step 3 — Add deployment link
Include the deployed application URL in your README.md file.
